In [1]:
import xml.etree.ElementTree as ET
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm_notebook
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
from time import time
import numpy as np
import sparsesvd

Parsing XML file

In [2]:
def parsing(filename):
    pages = ET.parse(filename).getroot()
    texts = [page.text for page in pages]
    morph = MorphAnalyzer()
    
    def process_text(text):
        # Removing punctuation, numbers and processing to lower case
        text = re.sub('[^а-яА-Я\s]', '', text).lower()
        words = text.split()
        
        # Removing stop words
        russian_stopwords = stopwords.words('russian')
#         english_stopwords = stopwords.words('english')
        
        stops = set(russian_stopwords)
        
        words = [morph.parse(word)[0].normal_form for word in words if not word in stops]
        return ' '.join(words)
    
    return [process_text(text) for text in tqdm_notebook(texts)]

file = 'wikipedia_2000_dump.xml'
texts = parsing(file)
print(texts[0][:995])

Widget Javascript not detected.  It may not be installed or enabled properly.



литва официальный название литовский республика государство географически расположить северный европа прибалтика столица страна вильнюс расположить восточный побережье балтийский море север граничить латвия юговосток белоруссия югозапад польша калининградский область россия член оон год ес нато год входить шенгенский зона еврозона поверхность равнинный след древний оледенение поль луга занимать территория леса кустарник болото внутренний вода высокий точка метр уровень море холм аукштояс аукштасис калнас юговосточный часть страна километр вильнюс крупный река неман нямунас вилий тысяча озр территория крупный друкшяй граница латвия литва белоруссия площадь километр самый глубокий таурагнас метр самый длинный асвеять длина километр местечко дубингяй климат переходный морской континентальный средний температура зимой минус летом выпадать мм осадок год полезный ископаемое торф минеральныематериал строительный материал территория современный литва заселить человек конец тысячелетие наш


Creating bag of words

In [3]:
start = time()
vectorizer = CountVectorizer(analyzer='word')

features = vectorizer.fit_transform(texts)
end = time()
print('elapsed time: {:.2f} s'.format(end - start))
print(features.shape)

elapsed time: 2.71 s
(2000, 128333)


In [4]:
sp_matrix = csc_matrix(features, dtype=np.float32)
_, singulars, _ = sparsesvd.sparsesvd(sp_matrix, sp_matrix.shape[1])
p_k = singulars / np.sum(singulars)
print('efficient dimension is about {:.2f}'.format(np.exp(-1 * np.sum(p_k * np.log(p_k)))))

efficient dimension is about 974.05
